<a href="https://colab.research.google.com/github/mattiadido95/Haze-Fog-suppression/blob/main/pix2pixHD_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install dominate gdown

In [2]:
from google.colab import drive
import os
import gdown

In [3]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
folder_path = "drive/MyDrive/Haze-Fog-suppression"
os.chdir(folder_path)

# Pix2PixHD download


In [ ]:
!git clone https://github.com/NVIDIA/pix2pixHD.git
os.chdir("pix2pixHD")

Cloning into 'pix2pixHD'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 55.68 MiB | 20.18 MiB/s, done.
Resolving deltas: 100% (156/156), done.
Updating files: 100% (115/115), done.
/content/drive/MyDrive/Haze-suppression/pix2pixHD


In [ ]:
os.makedirs("./checkpoints/label2city_1024p/")
os.chdir("./checkpoints/label2city_1024p/")

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download'
output = 'latest_net_G.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download
To: /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/checkpoints/label2city_1024p/latest_net_G.pth
100%|██████████| 732M/732M [00:11<00:00, 62.1MB/s]


'latest_net_G.pth'

In [ ]:
os.chdir("../..")

# Pix2PixHD Train


In [5]:
%cd pix2pixHD/

/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD


In [29]:
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1000"
!python train.py --label_nc 0 --no_instance --name nebbia --dataroot ./datasets/nebbia --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: ./datasets/nebbia
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
use_dropo

# Pix2Pix test

In [ ]:
#se non è stato eseguito il download
if os.getcwd() != "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD":
  os.chdir("pix2pixHD")

!ls ./checkpoints/label2city_1024p
#!./scripts/test_1024p.sh
!python test.py --name label2city_1024p --netG local --ngf 32 --resize_or_crop none

latest_net_G.pth  loss_log.txt
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 35
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: label2city_1024p
nef: 16
netG: local
ngf: 32
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatas